# Sizing script for a rocket engine

## Introduction
This script is used to size the Eve engine.
The script is designed to be used in a Jupyter Notebook.

## Running the script
To run the script, simply run all cells in the notebook.

## Imports
Let's start by importing the necessary libraries.

In [50]:
from math import sqrt
import cantera as ct
import matplotlib.pyplot as plt

## Constants

In [51]:
g0 = 9.80665  # m/s^2
R = 287.05  # J/kg/K
gamma = 1.4  # Cp/Cv

## Design parameters
The next cell contains the design parameters for the engine.
We start from a thrust target and a chamber pressure target.

In [52]:
# Main design parameters
thrust_target = 100  # N
chamber_pressure = 5e6  # Pa
# I_sp = 260  # s

# Nozzle parameters
atm_pressure = 101325  # Pa
exit_pressure = atm_pressure  # Pa

# Propellant parameters
mixture_ratio = 2.2  # oxidizer/fuel
fuel_species = 'CH4'
oxidizer_species = 'O2'
propellant_temperature = 300  # K

In [53]:
gas = ct.Solution('gri30.yaml')
gas.TP = propellant_temperature, chamber_pressure
gas.set_equivalence_ratio(phi=1/mixture_ratio, fuel=fuel_species, oxidizer=oxidizer_species)
gas.equilibrate('HP')

# Get properties from gas object
print('Gas properties:')
print('Density: {:.3f} kg/m^3'.format(gas.density))
print('Molecular weight: {:.3f} kg/kmol'.format(gas.mean_molecular_weight))
print('Cp: {:.3f} J/kg/K'.format(gas.cp_mass))
print('Cv: {:.3f} J/kg/K'.format(gas.cv_mass))
gamma_exhaust = gas.cp_mass / gas.cv_mass
print(f'Gamma: {gamma_exhaust:.3f}')

print(f"Adiabatic flame temperature: {gas.T:.1f} K")

fuel_density = gas.density * gas[fuel_species].molecular_weights[0]  # kg/m^3
oxidizer_density = gas.density * gas[oxidizer_species].molecular_weights[0]  # kg/m^3

# Calculate Isp
I_sp = g0 * gamma_exhaust / R * gas.T * (1 - (exit_pressure / chamber_pressure) ** ((gamma_exhaust - 1) / gamma_exhaust))
print(f"Isp: {I_sp:.1f} s")

Gas properties:
Density: 5.343 kg/m^3
Molecular weight: 27.972 kg/kmol
Cp: 1738.749 J/kg/K
Cv: 1441.505 J/kg/K
Gamma: 1.206
Adiabatic flame temperature: 3148.4 K
Isp: 63.1 s


## Propellant exit conditions

In [54]:
# Propellant exhaust conditions
exhaust_speed = I_sp * g0  # m/s
print(f"Exhaust speed: {exhaust_speed:.2f} m/s")

Exhaust speed: 619.00 m/s


## Flow rates
The mass flow rate is calculated from the thrust and the chamber pressure.

In [55]:
mass_flow_rate = thrust_target / I_sp / g0
print(f"Mass flow rate: {mass_flow_rate:.3f} kg/s")

# Calculate oxidizer/fuel mass ratio
fuel_mass_flow_rate = mass_flow_rate / (mixture_ratio + 1)
oxidizer_mass_flow_rate = fuel_mass_flow_rate * mixture_ratio

print(f"Oxidizer mass flow rate: {oxidizer_mass_flow_rate:.3f} kg/s \t {oxidizer_mass_flow_rate*1000:.3f} g/s")
print(f"Fuel mass flow rate: {fuel_mass_flow_rate:.3f} kg/s \t {fuel_mass_flow_rate*1000:.3f} g/s")

# Calculate oxidizer/fuel volume ratio
oxidizer_volume_flow_rate = oxidizer_mass_flow_rate / oxidizer_density
fuel_volume_flow_rate = fuel_mass_flow_rate / fuel_density

print(f"Oxidizer volume flow rate: {oxidizer_volume_flow_rate:.6f} m^3/s \t {oxidizer_volume_flow_rate*1000:.3f} L/s")
print(f"Fuel volume flow rate: {fuel_volume_flow_rate:.6f} m^3/s \t {fuel_volume_flow_rate*1000:.3f} L/s")

Mass flow rate: 0.162 kg/s
Oxidizer mass flow rate: 0.111 kg/s 	 111.066 g/s
Fuel mass flow rate: 0.050 kg/s 	 50.485 g/s
Oxidizer volume flow rate: 0.000650 m^3/s 	 0.650 L/s
Fuel volume flow rate: 0.000589 m^3/s 	 0.589 L/s
